In [ ]:
import gensim

import nltk
#nltk.download('stopwords')
import pandas as pd
import csv
import re
import googletrans
from googletrans import Translator

from string import punctuation
from nltk import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups

In [ ]:
df= pd.read_csv('query_result_2020-09-11T11_40_36.049226Z.csv')
type(df['post_description'])

## Translating to English

In [ ]:
translator= Translator()
translations= {}
unique_elements= df['post_description'].unique()

for element in unique_elements:
    translations[element] = translator.translate(element).text
#translations

In [ ]:
trans= translations.values()
#trans
df['post_description_en']= trans
df['post_description_en']= df['post_description_en'].str.lower()
df2= df['post_description_en'].str.replace(r'\W', ' ') #to remove any undesired special characters
#df2

## Data Preprocessing

In [ ]:
post_des= df2.tolist()
#post_des
post_preprocess= []
for i in post_des:
    post_preprocess.append(gensim.utils.simple_preprocess(i, deacc= True))
#post_preprocess

In [ ]:
texts= post_des
stops= set(stopwords.words('english'))
translate_tab = {ord(p): u" " for p in punctuation}

############### defining process_texts function ##############
def process_texts(texts):
    texts= [[word for word in line if word not in stops] for line in texts]
    #texts= texts.translate(translate_tab)
    #tokens = [token.strip() for token in tokenizer.tokenize(texts)]
    #tokens = [token for token in tokens if token not in eng_stopwords]
    #stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return texts

In [ ]:
train_texts= process_texts(post_preprocess)
#train_texts

## Data Cleaning

In [ ]:
texts2= []
for token in train_texts:
    t= []
    for word in token:
        k= False
        for i in word:
            if(ord(i)<65 & ord(i)>97):
                k= True
                break
        if(k== True):
            continue
        else:
            t.append(word)
    texts2.append(t)
#texts2

## Topic Modelling

In [ ]:
from gensim.corpora import Dictionary

newsgroups = fetch_20newsgroups()
eng_stopwords = set(stopwords.words('english'))

tokenizer = RegexpTokenizer(r'\s+', gaps=True)
stemmer = PorterStemmer()
translate_tab = {ord(p): u" " for p in punctuation}


def text2tokens(raw_text):
    """Convert a raw text to a list of stemmed tokens."""
    clean_text = raw_text.lower().translate(translate_tab)
    tokens = [token.strip() for token in tokenizer.tokenize(clean_text)]
    tokens = [token for token in tokens if token not in eng_stopwords]
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return [token for token in stemmed_tokens if len(token) > 2]  # skip short tokens

dataset = [text2tokens(txt) for txt in newsgroups['data']]  # convert a documents to list of tokens

dictionary2 = Dictionary(documents=dataset, prune_at=None)
dictionary2.filter_extremes(no_below=5, no_above=0.3, keep_n=None)  # use Dictionary to remove un-relevant tokens
dictionary2.compactify()

d2b_dataset = [dictionary2.doc2bow(doc) for doc in dataset]
corpus2= d2b_dataset

In [ ]:
from gensim.models import LdaModel
from gensim.models.wrappers import LdaMallet
#from gensim.corpora import Dictionary

dictionary= Dictionary(texts)
corpus= [dictionary2.doc2bow(text) for text in texts]

#print(dictionary)

In [ ]:
from gensim.models import CoherenceModel

def topic():
    score= {}

    for num_topic in range(3, 20):
        lda_model= LdaModel(corpus= corpus2, num_topics= num_topic, id2word= dictionary2)
        
        ################# coherence score ########################
        coherence_model_lda = CoherenceModel(
           model=lda_model, texts=dataset, dictionary= dictionary2, coherence='c_v'
        )
        coherence_lda = coherence_model_lda.get_coherence()
        #print('\nCoherence Score: ', coherence_lda)
        score[num_topic]= coherence_lda
    return sorted(score.items(), key= lambda x:x[1])[-1][0], score
topic()

In [ ]:
lda_model= LdaModel(corpus= corpus2, num_topics= 11, id2word= dictionary2)
lda_model.show_topics()

## Measuring Probability of a news to be categorized into correct topic

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(lda_model, corpus2, dictionary2)

In [ ]:
pred= []
prob= []
for i, row in enumerate(lda_model[corpus]):
   row = sorted(row, key=lambda x: (x[1]), reverse=True)
   print(row)
   pred.append(row[0][0])
   prob.append(row[0][1])

In [ ]:
df['prob'] = prob
df['cluster'] = pred
df[['post_description_en', 'cluster']]